In [ ]:
import arcticdb as adb
import json
import numpy as np
import pandas as pd
import os
from tradingo.utils import get_instruments, get_config, with_instrument_details

In [ ]:
a = adb.Arctic("lmdb:///home/rory/dev/airflow/test/arctic.db")

In [ ]:
instruments = get_instruments(get_config())

orig_prices = a.get_library("ASSET_PRICES").read("ETFT.ADJ_CLOSE.yfinance")
start_dates = orig_prices.data.apply(lambda i: i.first_valid_index()).rename("start_date")
instruments = instruments.merge(start_dates, left_index=True, right_index=True, how='left')

In [ ]:
PRICES = with_instrument_details((orig_prices.data[
                instruments[(instruments.start_date < pd.Timestamp("2018-03-01"))].index
          ]).dropna(), instruments, columns=["Asset Class"])
RETURNS = np.log(PRICES/PRICES.shift()).dropna()

In [ ]:
PRICES.dropna()

In [ ]:
PRICES.groupby(["Asset Class"], axis=1).mean().plot()

In [ ]:
instruments

In [ ]:
corr = PRICES.pct_change().groupby("Asset Class", axis=1).corr().abs()#.unstack().sort_values()

In [ ]:
(1+RETURNS.loc[:,"Commodities & Digital Assets"].transpose().merge(instruments.Name, left_index=True, right_index=True).reset_index(drop=True).set_index(["Name"]).transpose()).cumprod().plot()

In [ ]:
corr[corr.between(0.3, 0.6)]

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

In [ ]:
pca.fit(RETURNS)

In [ ]:
pd.DataFrame(pca.components_, index=RETURNS.columns)

In [ ]:
df = pd.DataFrame(pca.components_, columns=RETURNS.columns)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
lib = a.get_library("BACKTEST")

In [ ]:
inst_returns = with_instrument_details(lib.read("ETFT.trend.INSTRUMENT_RETURNS").data, instruments=instruments, columns=["Asset Class"])

In [ ]:
inst_returns